# 1D Inversion for Multiple Soundings

In [9]:
import numpy as np
import matplotlib.pyplot as plt

from discretize import TensorMesh
    
import SimPEG.electromagnetics.frequency_domain as fdem

from SimPEG import (
    maps,
    data,
    data_misfit,
    inverse_problem,
    regularization,
    optimization,
    directives,
    inversion,
)

### Load Data and Plot

In [10]:
# path to the directory containing our data
dir_path = ".\Data" # or something else
filename = "em1dfm_data.txt" # or something else

# files to work with
data_filename = dir_path + filename

In [ ]:
background_conductivity = 1 # estimated soil conductivity

imported_1d_data = np.loadtxt(str(data_filename), skiprows=1)

### Read from the file ###
# for example:
# frequencies = imported_1d_data[]
# x_positions = imported_1d_data[]
# y_positions = imported_1d_data[]

In [ ]:
# This will depend on how the data is structured in the data file
def get_dobs_from_1d_data(index):
    return 0

def generate_survey(frequencies, x, y, moment=1):
    
    data_type = "ppm"

    # Defining transmitter locations
    source_location = np.c_[x, y, 0.1]
    source_orientation = "z" #could change

    # Define receiver locations
    receiver_location = np.c_[x+1, y, 0.1]
    receiver_orientation = "z" #could change

    source_list = [] 
    # loop over all sources
    for freq in frequencies:
        # Define receivers that measure real and imaginary component
        # magnetic field data in ppm.
        receiver_list = []
        receiver_list.append(
            fdem.receivers.PointMagneticFieldSecondary(
                receiver_location,
                orientation=receiver_orientation,
                data_type=data_type,
                component="real",
            )
        )
        receiver_list.append(
            fdem.receivers.PointMagneticFieldSecondary(
                receiver_location,
                orientation=receiver_orientation,
                data_type=data_type,
                component="imag",
            )
        )

        # Define a magnetic dipole source at each frequency
        source_list.append(
            fdem.sources.MagDipole(
                receiver_list=receiver_list,
                frequency=freq,
                location=source_location,
                orientation=source_orientation,
                moment=moment,
            )
        )

    # Define the FDEM survey
    survey = fdem.survey.Survey(source_list)
    return survey

# Parameters here should be changed to find the ideal inversion condition
def define_inverse_problem_1DLayered(data_object, regularization_mesh, simulation, reference_model, param):
    
    dmis_L2 = data_misfit.L2DataMisfit(simulation=simulation, data=data_object)
    
    reg_L2 = regularization.WeightedLeastSquares(
        regularization_mesh,
        length_scale_x=10.0,
        reference_model=reference_model,
        reference_model_in_smooth=False,
    )
    
    # There are other parameters withint InexactFaussNewton we can change as well
    opt_L2 = optimization.InexactGaussNewton(
        maxIter=100, maxIterLS=20, maxIterCG=20, tolCG=1e-3
    )

    inv_prob_L2 = inverse_problem.BaseInvProblem(dmis_L2, reg_L2, opt_L2)
    
    update_jacobi = directives.UpdatePreconditioner(update_every_iteration=True)
    starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=param["beta0_ratio"])
    beta_schedule = directives.BetaSchedule(coolingFactor=param["coolingFactor"], coolingRate=param["coolingRate"])
    target_misfit = directives.TargetMisfit(chifact=param["chifact"])

    directives_list_L2 = [update_jacobi, 
                        starting_beta, 
                        beta_schedule, 
                        target_misfit]
    
    inv_L2 = inversion.BaseInversion(inv_prob_L2, directives_list_L2)

    return inv_L2

In [ ]:
indx = 0

dobs = get_dobs_from_1d_data(indx)

# Plot FEM response for all frequencies
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.loglog(frequencies, abs(dobs[0::2]), "r-o", lw=3)
ax.loglog(frequencies, abs(dobs[1::2]), "b:o", lw=3)
ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("|Bz Secondary| [T]")
ax.set_title(f"Magnetic Field Data at x={x_positions[indx]} and y={y_positions[indx]}")
ax.legend(["Real", "Imaginary"])
plt.grid()
plt.show()

## Weighted Least-Square Inversion

### Define a Regularization Mesh, Reference Model and the Mapping

In [ ]:
# Defind layer thicknesses
depth_min = 0.1  # top layer thickness
depth_max = 3.0  # depth to lowest layer
geometric_factor = 1.2  # rate of thickness increase
layer_thicknesses = [depth_min]
while np.sum(layer_thicknesses) < depth_max:
    layer_thicknesses.append(geometric_factor * layer_thicknesses[-1])

# Define 1D cell widths and create regularization mesh
h = np.r_[layer_thicknesses, layer_thicknesses[-1]]
h = np.flipud(h)
regularization_mesh = TensorMesh([h], "N")

# Define Starting model
n_layers = len(layer_thicknesses) + 1
starting_model = np.log((background_conductivity) * np.ones(n_layers))

# Define mapping
log_resistivity_map = maps.ExpMap(nP=n_layers)

# Reference model is also log-resistivity values (S/m)
reference_resistivity_model = starting_model.copy()

## Perform Inversions on Soundings at All Positions

In [7]:
# Set the inversion parameters
param = {
    "beta0_ratio": 1e1,
    "coolingFactor": 2.0, 
    "coolingRate": 3.0, 
    "chifact": 1
}

In [ ]:
recovered_models = []

for i, x in enumerate(x_positions):
    for j, y in enumerate(y_positions):
        # Generate a survey at the location
        survey = generate_survey(frequencies=frequencies, x_position=x, y_postion=y)

        # Get the data at the desired location
        dobs = get_dobs_from_1d_data(imported_1d_data, location_index=i)
        ## We need to look deeper into how to assign uncertainty here
        uncertainties = 0.05 * np.abs(dobs) * np.ones(np.shape(dobs))
        data_object = data.Data(survey, dobs=dobs, noise_floor=uncertainties)

        # Define the L2 simulation
        simulation_L2 = fdem.Simulation1DLayered(
        survey=survey, thicknesses=layer_thicknesses, rhoMap=log_resistivity_map
        )

        # Define the Inversion
        inv_L2 = define_inverse_problem_1DLayered(simulation_L2, data_object, regularization_mesh, reference_resistivity_model, param)
        
        # Recover the model
        recovered_L2model = inv_L2.run(starting_model)
        recovered_models.append(recovered_L2model)


## Plot in 3D by Projecting 1D Data onto 3D mesh

## Interpolation between Each Recovered Models